# Tarea 3.1: Aprendizaje no supervisado

### Integrantes del grupo:
- José Pinto 202073559-K
- Vicente Ruiz 202073595-9

Los juegos de Pokémon son una serie de videojuegos desarrollados por Game Freak y publicados por Nintendo y The Pokémon Company. 

En estos juegos, los jugadores asumen el rol de entrenadores cuyo objetivo es capturar y entrenar criaturas llamadas "pokémon" para competir en batallas contra otros entrenadores y líderes de gimnasio. Los juegos se centran en la exploración, el combate estratégico y la recolección. Estos son los atributos básicos que se utilizan para calcular cuánto daño hará un ataque en los juegos. Este dataset trata sobre los juegos de Pokémon (NO sobre las cartas de Pokémon ni Pokémon Go).

Las variables incluidas en el dataset reflejan las características esenciales de los pokémon que son relevantes para el juego:

1. #(Número): Identificación única en la Pokédex.

2. Name (Nombre): Nombre del pokémon.

3. Type 1 (Tipo 1): Tipo principal, determina fortalezas y debilidades.

4. Type 2 (Tipo 2): Tipo secundario, si lo tiene.

5. Total: Suma de todas las estadísticas base.

6. HP: Puntos de salud, vitalidad en batalla.

7. Attack (Ataque): Fuerza en ataques físicos.

8. Defense (Defensa): Resistencia a ataques físicos.

9. Sp. Atk (Ataque Especial): Fuerza en ataques especiales.

10. Sp. Def (Defensa Especial): Resistencia a ataques especiales.

11. Speed (Velocidad): Determina el orden de ataque en batalla.

12. Generation (Generación): Juego en el que debutó.

13. Legendary (Legendario): Indica si es legendario, afectando su rareza y habilidades únicas.


KMEANS - KMEDIANS - KMEDOIDS

### 1.- Cargamos y verificamos integridad de los datos. Realizar curación si corresponde.

In [577]:
import pandas as pd


# Mostramos las primeras filas del dataset
df = pd.read_csv('Pokemon.csv')
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [578]:
# Verificar el tamaño del dataset
print(df.shape)

(800, 13)


In [579]:
# Verificar los tipos de datos de cada columna
print("Tipos de datos:\n", df.dtypes)

Tipos de datos:
 #              int64
Name          object
Type 1        object
Type 2        object
Total          int64
HP             int64
Attack         int64
Defense        int64
Sp. Atk        int64
Sp. Def        int64
Speed          int64
Generation     int64
Legendary       bool
dtype: object


In [580]:
# Resumen estadístico de las columnas numéricas
print("Resumen estadístico:\n", df.describe())

Resumen estadístico:
                 #      Total          HP      Attack     Defense     Sp. Atk   
count  800.000000  800.00000  800.000000  800.000000  800.000000  800.000000  \
mean   362.813750  435.10250   69.258750   79.001250   73.842500   72.820000   
std    208.343798  119.96304   25.534669   32.457366   31.183501   32.722294   
min      1.000000  180.00000    1.000000    5.000000    5.000000   10.000000   
25%    184.750000  330.00000   50.000000   55.000000   50.000000   49.750000   
50%    364.500000  450.00000   65.000000   75.000000   70.000000   65.000000   
75%    539.250000  515.00000   80.000000  100.000000   90.000000   95.000000   
max    721.000000  780.00000  255.000000  190.000000  230.000000  194.000000   

          Sp. Def       Speed  Generation  
count  800.000000  800.000000   800.00000  
mean    71.902500   68.277500     3.32375  
std     27.828916   29.060474     1.66129  
min     20.000000    5.000000     1.00000  
25%     50.000000   45.000000     2.0

In [581]:
# Verificamos valores faltantes
print("Valores faltantes por columna:\n", df.isnull().sum())

Valores faltantes por columna:
 #               0
Name            0
Type 1          0
Type 2        386
Total           0
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64


In [582]:
# Eliminar la columna '#', 'Name', 'Generation', 'Legendary', 'Type 1' y 'Type 2'
df = df.drop(['#', 'Name', 'Generation', 'Legendary', 'Type 1', 'Type 2'], axis=1)

In [583]:
# Verificamos valores faltantes
print("Valores faltantes por columna:\n", df.isnull().sum())

Valores faltantes por columna:
 Total      0
HP         0
Attack     0
Defense    0
Sp. Atk    0
Sp. Def    0
Speed      0
dtype: int64


### 2.- Graficamos la matriz de correlación y realizamos un scatter_matrix con plotly.

In [584]:
import plotly.express as px

# Calcular la matriz de correlación con columnas numéricas
correlation_matrix = df.corr()

# Graficar la matriz de correlación
fig = px.imshow(correlation_matrix, 
                labels=dict(x="Atributos", y="Atributos", color="Correlación"), 
                x=correlation_matrix.columns, 
                y=correlation_matrix.columns,
                color_continuous_scale=px.colors.diverging.RdBu,
                zmin=-1, zmax=1)

fig.update_layout(title="Matriz de Correlación de los Atributos de Pokémon")
fig.show()

Podemos notar que el atributo Total tiene una alta correlación con los atributos, esto se puede deber a que el atributo Total es la suma de los atributos HP, Attack, Defense, Sp. Atk, Sp. Def y Speed. Al no saber si esto puede afectar el resultado de los algoritmos de clustering, se decide eliminar el atributo Total.

In [585]:
# Eliminar la columna 'Total'
df = df.drop(['Total'], axis=1)


# Calcular la matriz de correlación
correlation_matrix_no_total = df.corr()

# Graficar la matriz de correlación
fig = px.imshow(correlation_matrix_no_total, 
                labels=dict(x="Atributos", y="Atributos", color="Correlación"), 
                x=correlation_matrix_no_total.columns, 
                y=correlation_matrix_no_total.columns,
                color_continuous_scale=px.colors.diverging.RdBu,
                zmin=-1, zmax=1)


fig.update_layout(title="Matriz de Correlación de los Atributos de Pokémon (sin Total)")
fig.show()

In [586]:
import plotly.express as px
from plotly.subplots import make_subplots

#  scatter_matrix sin numeros en los ejes de las gráficas
fig = px.scatter_matrix(df, labels={col:col for col in df.columns}, height=800)
fig.update_traces(diagonal_visible=False)
fig.show()


### 3.- Seleccionamos los valores del dataset que son de utilidad para entrenar cada modelo.

Al observar la matriz de correlación, decidimos seleccionar los siguientes atributos para cada modelo:

- K-means: Se eligen "Attack" y "Sp. Atk" debido a su alta correlación, lo que favorece la formación de clusters basados en la media aritmética como centroide.

- K-medians: Se eligen "HP" y "Speed" por su baja correlación con otras características, lo que facilita la formación de clusters utilizando la mediana como centroide y su robustez ante valores atípicos.

- K-medoids: Se seleccionan "Sp. Atk" y "Sp. Def" debido a su alta correlación, ya que K-medoids busca una muestra real representativa para cada cluster.

In [587]:
#  scatter_matrix para atack y sp.atk

fig = px.scatter(df, x='Attack', y='Sp. Atk', title="Ataque vs Ataque Especial")
fig.update_layout(title="Ataque vs Ataque Especial")

#  scatter_matrix para hp y speed
fig2 = px.scatter(df, x='HP', y='Speed', title="Puntos de Vida vs Velocidad")
fig2.update_layout(title="Puntos de Vida vs Velocidad")


# scatter_matrix para sp atk y sp def
fig3 = px.scatter(df, x='Sp. Atk', y='Sp. Def', title="Ataque Especial vs Defensa Especial")
fig3.update_layout(title="Ataque Especial vs Defensa Especial")

fig = make_subplots(rows=2, cols=2, subplot_titles=("Ataque vs Ataque Especial", "Puntos de Vida vs Velocidad", "Ataque Especial vs Defensa Especial"))

fig.add_trace(fig2['data'][0], row=1, col=1)
fig.add_trace(fig['data'][0], row=1, col=2)
fig.add_trace(fig3['data'][0], row=2, col=1)

fig.update_layout(title="Gráficos de dispersión de los atributos de Pokémon")
fig.show()


### 4.- Entrenamos un modelo de KMEANS - KMEDIANS - KMEDOIDS

#### 4.1. Mediante el método del codo, determinamos el número de clústeres óptimo, explicar por que lo eligieron, con el gráfico correspondiente.

In [588]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn_extra.cluster import KMedoids

import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [589]:
# Seleccionar las columnas 'Attack' y 'Attack Special'
X_kmeans = df[['Attack', 'Sp. Atk']].values
# Seleccionar las columnas 'HP' y 'Speed'
X_kmedians = df[['HP', 'Speed']].values
# Seleccionar las columnas 'Sp. Atk' y 'Sp. Def'
X_kmedoids = df[['Sp. Atk', 'Sp. Def']].values

In [590]:
# Calcular la inercia para kmeans
def elbow_method_kmeans(X):
    distortions = []
    K = range(1, 11)
    for k in K:
        kmeans = KMeans(n_clusters=k, random_state=0)
        kmeans.fit(X)
        distortions.append(kmeans.inertia_)
    return distortions

# Calcular la inercia para kmedians
def kmedians(X, k):
    medians = X[np.random.choice(X.shape[0], k, replace=False)]
    for _ in range(100):  # número máximo de iteraciones
        labels = np.argmin(cdist(X, medians, metric='cityblock'), axis=1)
        new_medians = np.array([np.median(X[labels == i], axis=0) for i in range(k)])
        if np.all(medians == new_medians):
            break
        medians = new_medians
    return medians, labels

def elbow_method_kmedians(X):
    distortions = []
    K = range(1, 11)
    for k in K:
        medians, labels = kmedians(X, k)
        distortion = np.sum(np.min(cdist(X, medians, 'cityblock'), axis=1))
        distortions.append(distortion)
    return distortions

# Calcular la inercia para kmedoids
def elbow_method_kmedoids(X):
    distortions = []
    K = range(1, 11)
    for k in K:
        kmedoids = KMedoids(n_clusters=k, random_state=0)
        kmedoids.fit(X)
        distortions.append(kmedoids.inertia_)
    return distortions

In [591]:
# Calcular la inercia para kmeans
distortions_kmeans = elbow_method_kmeans(X_kmeans)

# Calcular la inercia para kmedians
distortions_kmedians = elbow_method_kmedians(X_kmedians)

# Calcular la inercia para kmedoids
distortions_kmedoids = elbow_method_kmedoids(X_kmedoids)

# graficar la inercia para kmeans, kmedians y kmedoids con subplots
fig = px.line(x=range(1, 11), y=distortions_kmeans, title="Método del codo para KMeans")
fig.update_layout(title="Método del codo para KMeans")

fig2 = px.line(x=range(1, 11), y=distortions_kmedians, title="Método del codo para KMedians")
fig2.update_layout(title="Método del codo para KMedians")

fig3 = px.line(x=range(1, 11), y=distortions_kmedoids, title="Método del codo para KMedoids")
fig3.update_layout(title="Método del codo para KMedoids")

figf = make_subplots(rows=2, cols=2, subplot_titles=("KMeans", "KMedians", "KMedoids"))
figf.add_trace(fig2['data'][0], row=1, col=1)
figf.add_trace(fig['data'][0], row=1, col=2)
figf.add_trace(fig3['data'][0], row=2, col=1)

figf.update_layout(title="Método del codo para KMeans, KMedians y KMedoids")
figf.show()

Al revisar los gráficos de codo, se observa que el número óptimo de clusters es 5 para K-means y K-medians. Pero 4 para K-medoids. Esto se debe a que en K-means, K-medians y K-medoids, el codo se encuentra en el punto donde la inercia disminuye de forma más pronunciada.

#### 4.2. Luego de determinar el número, entrenar con dicho número y realizar un grafico de radar para cada clúster que muestre todas las variables que utilizó para entrenar utilizando los centroides de cada algoritmo como valor representativo de cada clúster.

In [592]:
# Entrenar el modelo KMeans con 5 clusters
kmeans = KMeans(n_clusters=5, random_state=0)
kmeans.fit(X_kmeans)

# Agregar las etiquetas de los clusters kmeans al DataFrame
df['Cluster-KMeans'] = kmeans.labels_

# Calcular los centroides
centroids = kmeans.cluster_centers_
# Agrupar los centroides en un DataFrame
centroids_df = pd.DataFrame(centroids, columns=['Attack', 'Sp. Atk'])


# Grafico radar con x atacque y y ataque especial, con el centroide en el centro y datos de los clusters con diferentes colores
fig = px.scatter_polar(centroids_df, r='Attack', theta='Sp. Atk')
fig.add_trace(px.scatter_polar(df, r='Attack', theta='Sp. Atk', color='Cluster-KMeans').data[0])
fig.update_layout(title="Centroide y datos de los clusters Ataque vs Ataque Especial")
fig.update_polars(radialaxis=dict(visible=True))
fig.show()

In [593]:
# Entrenar el modelo KMedians con 5 clusters
medians, labels = kmedians(X_kmedians, 5)

# Agregar las etiquetas de los clusters al dataframe
df['Cluster-KMedians'] = labels

# Calcular los centroides
centroids = medians

# Agrupar los centroides en un DataFrame
centroids_df = pd.DataFrame(centroids, columns=['HP', 'Speed'])

# Grafico radar con x hp y y velocidad, con el centroide en el centro y datos de los clusters con diferentes colores
fig = px.scatter_polar(centroids_df, r='HP', theta='Speed')
fig.add_trace(px.scatter_polar(df, r='HP', theta='Speed', color='Cluster-KMedians').data[0])
fig.update_layout(title="Centroide y datos de los clusters Puntos de Vida vs Velocidad")
# Mostrar ejes radiales
fig.update_polars(radialaxis=dict(visible=True))
fig.show()



In [594]:
# Entrenar el modelo KMedoids con 4 clusters
kmedoids = KMedoids(n_clusters=4, random_state=0)

kmedoids.fit(X_kmedoids)

# Agregar las etiquetas de los clusters al dataframe
df['Cluster-KMedoids'] = kmedoids.labels_

# Calcular los centroides
centroids = kmedoids.cluster_centers_

# Agrupar los centroides en un DataFrame
centroids_df = pd.DataFrame(centroids, columns=['Sp. Atk', 'Sp. Def'])

# Grafico radar con x ataque especial y y defensa especial, con el centroide en el centro y datos de los clusters con diferentes colores
fig = px.scatter_polar(centroids_df, r='Sp. Atk', theta='Sp. Def')
fig.add_trace(px.scatter_polar(df, r='Sp. Atk', theta='Sp. Def', color='Cluster-KMedoids').data[0])
fig.update_layout(title="Centroide y datos de los clusters Ataque Especial vs Defensa Especial")
# Mostrar ejes radiales
fig.update_polars(radialaxis=dict(visible=True))
fig.show()

#### 4.3. Realizamos un grafico scatter_matrix con plotly usando como parámetro color la variable clústeres.

In [595]:
fig1= px.scatter(df, x='Attack', y='Sp. Atk', color='Cluster-KMeans', title="KMeans")
fig1.update_layout(title="KMeans")

fig2 = px.scatter(df, x='HP', y='Speed', color='Cluster-KMedians', title="KMedians")
fig2.update_layout(title="KMedians")

fig3 = px.scatter(df, x='Sp. Atk', y='Sp. Def', color='Cluster-KMedoids', title="KMedoids")
fig3.update_layout(title="KMedoids")

figf = make_subplots(rows=2, cols=2, subplot_titles=("KMeans", "KMedians", "KMedoids"))
figf.add_trace(fig1['data'][0], row=1, col=1)
figf.add_trace(fig2['data'][0], row=1, col=2)
figf.add_trace(fig3['data'][0], row=2, col=1)

figf.update_layout(title="Gráficos de dispersión de los atributos de Pokémon con KMeans, KMedians y KMedoids")
figf.show()

In [596]:
# scatter_matrix con los clusters de kmeans
columns_to_include = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']


fig = px.scatter_matrix(df, dimensions=columns_to_include, color='Cluster-KMeans', height=600)
fig.update_traces(diagonal_visible=False)
fig.update_layout(title="Matriz de dispersión de los atributos de Pokémon con KMeans")
fig.show()

In [597]:
# scatter_matrix con los clusters de kmedians
fig2 = px.scatter_matrix(df, dimensions=columns_to_include, color='Cluster-KMedians', height=600)
fig2.update_traces(diagonal_visible=False)
fig2.update_layout(title="Matriz de dispersión de los atributos de Pokémon con KMedians")
fig2.show()

In [598]:
# scatter_matrix con los clusters de kmedoids
fig3 = px.scatter_matrix(df, dimensions=columns_to_include, color='Cluster-KMedoids', height=600)
fig3.update_traces(diagonal_visible=False)

fig3.update_layout(title="Matriz de dispersión de los atributos de Pokémon con KMedoids")
fig3.show()


#### 4.4 Tomamos dos variables y graficamos con una X el punto del centroide, medoide y median, según corresponda.

In [605]:
# 4.4 Tomar dos variables que prefieran (pero las mismas para comparar los 3 modelos) y graficar con una X el punto del centroide, medoide y median, según corresponda.

# Seleccionar las columnas 'Attack' y 'Sp. Atk'
X_kmeans = df[['Attack', 'Sp. Atk']].values
# Seleccionar las columnas 'Attack' y 'Sp. Atk'
X_kmedians = df[['Attack', 'Sp. Atk']].values
# Seleccionar las columnas 'Attack' y 'Sp. Atk'
X_kmedoids = df[['Attack', 'Sp. Atk']].values

# Entrenar el modelo KMeans con 5 clusters
kmeans = KMeans(n_clusters=5, random_state=0)
kmeans.fit(X_kmeans)

# Calcular los centroides
centroids = kmeans.cluster_centers_

# Agregamos los centroides kmeans en un DataFrame
centroids_df = pd.DataFrame(centroids, columns=['Attack', 'Sp. Atk'])
centroids_df['Centroid'] = 'KMeans'

# Entrenar el modelo KMedians con 5 clusters
medians, labels = kmedians(X_kmedians, 5)

# Calcular los centroides
centroids = medians

# Agrupar los centroides en un DataFrame
centroids_df2 = pd.DataFrame(centroids, columns=['Attack', 'Sp. Atk'])
centroids_df2['Centroid'] = 'KMedians'

# Entrenar el modelo KMedoids con 4 clusters
kmedoids = KMedoids(n_clusters=4, random_state=0)

kmedoids.fit(X_kmedoids)

# Calcular los centroides
centroids = kmedoids.cluster_centers_

# Agrupar los centroides en un DataFrame
centroids_df3 = pd.DataFrame(centroids, columns=['Attack', 'Sp. Atk'])
centroids_df3['Centroid'] = 'KMedoids'

# Graficar los centroides de los modelos KMeans, KMedians y KMedoids
fig = px.scatter(df, x='Attack', y='Sp. Atk', title="Ataque vs Ataque Especial")

fig.add_trace(go.Scatter(
    x=centroids_df['Attack'],
    y=centroids_df['Sp. Atk'],
    mode='markers',
    marker=dict(color='red', size=10, symbol='x'),
    name='KMeans Centroid'
))


fig.add_trace(go.Scatter(
    x=centroids_df2['Attack'],
    y=centroids_df2['Sp. Atk'],
    mode='markers',
    marker=dict(color='green', size=10, symbol='x'),
    name='KMedians Centroid'
))


fig.add_trace(go.Scatter(
    x=centroids_df3['Attack'],
    y=centroids_df3['Sp. Atk'],
    mode='markers',
    marker=dict(color='black', size=10, symbol='x'),
    name='KMedoids Centroid'
))

fig.update_layout(title="Ataque vs Ataque Especial")
fig.show()